Load data for coasters and user preferences

In [77]:
%reset

import psycopg2
import pandas as pd


def get_db_connection():
    try:
        conn = psycopg2.connect(
            host='cp-ai.cbsscwgeqp5j.us-east-2.rds.amazonaws.com',
            port=5432,
            database='postgres',
            user='postgres',
            password='CedarP0int'
        )
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL database: {e}")
        return None

# Example usage:
conn = get_db_connection()
roller_coaster_df = pd.read_sql("SELECT * FROM ROLLER_COASTERS", conn)
user_ratings_df = pd.read_sql("SELECT * FROM USER_ROLLER_COASTER_RATINGS", conn)

conn.close()
roller_coaster_df = roller_coaster_df.set_index('id')
original_roller_coaster_ratings = roller_coaster_df.copy()


C:\Users\rober\AppData\Local\Temp\ipykernel_52380\2986927999.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  roller_coaster_df = pd.read_sql("SELECT * FROM ROLLER_COASTERS", conn)
C:\Users\rober\AppData\Local\Temp\ipykernel_52380\2986927999.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_ratings_df = pd.read_sql("SELECT * FROM USER_ROLLER_COASTER_RATINGS", conn)


Show data loaded for verification

In [78]:
roller_coaster_df.head()

,name,url,amusement_park,type,design,status,manufacturer,model,length,height,...,imputed_length,imputed_height,imputed_drop,imputed_inversion_count,imputed_speed,imputed_vertical_angle,imputed_restraints,imputed_g_force,imputed_intensity,imputed_duration
id,,,,,,,,,,,,,,,,,,,,,
12271.0,Race,/12271.htm,Wiener Prater,Steel,Sit Down,Operating,SBF Visa Group,F1 Vortex Coaster (MX607),NaN,NaN,...,744.53,27.84,NaN,0.0,23.733863,NaN,T-Bar,NaN,Family,77.876923
11976.0,Racing Coaster,/11976.htm,Trampoline Trier,Steel,Sit Down,Operating,SBF Visa Group,F1 Vortex Coaster (MX607),NaN,NaN,...,744.53,27.84,NaN,0.0,23.733863,NaN,T-Bar,NaN,Family,77.876923
19966.0,Spinning Coaster,/19966.htm,Track Family Fun Parks,Steel,Sit Down,Operating,SBF Visa Group,MX608,NaN,NaN,...,187.50,14.00,NaN,0.0,23.733863,NaN,T-Bar,NaN,Family,77.876923
4288.0,Roller Coaster,/4288.htm,Papéa Parc,Steel,Sit Down,Operating,Zierer,Large,1181.1,26.3,...,1181.10,26.30,24.99,0.0,22.400000,NaN,T-Bar,NaN,Family,77.876923
6536.0,Bandit Bomber,/6536.htm,Yas Waterworld,Steel,Inverted,Operating,Vekoma,Splash Party,1765.1,75.5,...,1765.10,75.50,72.48,0.0,31.100000,NaN,Single position lap bar,NaN,Family,87.666667


In [79]:
user_ratings_df.head()

,user_id,roller_coaster_id,roller_coaster_name,amusement_park,rating
0,rwm70c,13377.0,Slinky Dog Dash,Walt Disney World - Disney's Hollywood Studios,NaN
1,rwm70c,295.0,Chip & Dale’s Gadget Coaster,Disneyland,NaN
2,rwm70c,15124.0,Woodstock Express,Michigan's Adventure,NaN
3,rwm70c,9720.0,Seven Dwarfs Mine Train,Walt Disney World - Magic Kingdom,NaN
4,rwm70c,15.0,Wilderness Run,Cedar Point,NaN


Define features of interest

In [80]:
# add an inversion count
roller_coaster_df['has_inversions'] = roller_coaster_df['inversion_count'] > 0

intensity_map = {
    'Kiddie': 0,
    'Family': 1,
    'Thrill': 2,
    'Extreme': 3
}
roller_coaster_df['intensity_level'] = roller_coaster_df['imputed_intensity'].map(intensity_map)
numeric_feature_columns = ['imputed_length','imputed_height','imputed_duration','imputed_speed']
cat_columns = ['type','design','manufacturer','imputed_restraints','imputed_intensity','imputed_duration','imputed_inversion_count']
cat_one_hot_encoded = pd.get_dummies(roller_coaster_df[cat_columns])
X = pd.concat([roller_coaster_df[numeric_feature_columns],roller_coaster_df['intensity_level'],cat_one_hot_encoded],axis=1 )
X.head()


,imputed_length,imputed_height,imputed_duration,imputed_speed,intensity_level,imputed_duration,imputed_inversion_count,type_Steel,type_Wood,design_Bobsled,...,imputed_restraints_Single position lap barHeadrest,imputed_restraints_Single position lap barHeadrestSeatbelt,imputed_restraints_Single position lap barSeatbelt,imputed_restraints_T-Bar,imputed_restraints_T-BarSeatbelt,imputed_restraints_UNKNOWN,imputed_intensity_Extreme,imputed_intensity_Family,imputed_intensity_Kiddie,imputed_intensity_Thrill
id,,,,,,,,,,,,,,,,,,,,,
12271.0,744.53,27.84,77.876923,23.733863,1,77.876923,0.0,True,False,False,...,False,False,False,True,False,False,False,True,False,False
11976.0,744.53,27.84,77.876923,23.733863,1,77.876923,0.0,True,False,False,...,False,False,False,True,False,False,False,True,False,False
19966.0,187.50,14.00,77.876923,23.733863,1,77.876923,0.0,True,False,False,...,False,False,False,True,False,False,False,True,False,False
4288.0,1181.10,26.30,77.876923,22.400000,1,77.876923,0.0,True,False,False,...,False,False,False,True,False,False,False,True,False,False
6536.0,1765.10,75.50,87.666667,31.100000,1,87.666667,0.0,True,False,False,...,False,False,False,False,False,False,False,True,False,False


Scale Values

In [81]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_numeric_scaled = pd.DataFrame(
    scaler.fit_transform(roller_coaster_df[numeric_feature_columns]),
    columns=numeric_feature_columns,
    index=roller_coaster_df.index
)
X_numeric_scaled.head()

,imputed_length,imputed_height,imputed_duration,imputed_speed
id,,,,
12271.0,-0.563310,-0.522306,-0.531509,-0.489559
11976.0,-0.563310,-0.522306,-0.531509,-0.489559
19966.0,-1.007444,-0.846453,-0.531509,-0.489559
4288.0,-0.215222,-0.558375,-0.531509,-0.579210
6536.0,0.250416,0.593939,-0.174804,0.005531


In [82]:
X_cleaned = X.drop(columns=X_numeric_scaled.columns)
X_final = pd.concat([X_numeric_scaled,X_cleaned], axis=1)
X_final.head()

,imputed_length,imputed_height,imputed_duration,imputed_speed,intensity_level,imputed_inversion_count,type_Steel,type_Wood,design_Bobsled,design_Flying,...,imputed_restraints_Single position lap barHeadrest,imputed_restraints_Single position lap barHeadrestSeatbelt,imputed_restraints_Single position lap barSeatbelt,imputed_restraints_T-Bar,imputed_restraints_T-BarSeatbelt,imputed_restraints_UNKNOWN,imputed_intensity_Extreme,imputed_intensity_Family,imputed_intensity_Kiddie,imputed_intensity_Thrill
id,,,,,,,,,,,,,,,,,,,,,
12271.0,-0.563310,-0.522306,-0.531509,-0.489559,1,0.0,True,False,False,False,...,False,False,False,True,False,False,False,True,False,False
11976.0,-0.563310,-0.522306,-0.531509,-0.489559,1,0.0,True,False,False,False,...,False,False,False,True,False,False,False,True,False,False
19966.0,-1.007444,-0.846453,-0.531509,-0.489559,1,0.0,True,False,False,False,...,False,False,False,True,False,False,False,True,False,False
4288.0,-0.215222,-0.558375,-0.531509,-0.579210,1,0.0,True,False,False,False,...,False,False,False,True,False,False,False,True,False,False
6536.0,0.250416,0.593939,-0.174804,0.005531,1,0.0,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False


Build user profile

In [89]:
# Step 1: Filter out unrated coasters
rated_only_df = user_ratings_df[user_ratings_df['rating'].notnull()]

# Step 2: Merge with coaster metadata
rated = rated_only_df.merge(roller_coaster_df, left_on='roller_coaster_id', right_on='id')

rated_features = X_final.loc[rated['roller_coaster_id']]
rated_features.head()

,imputed_length,imputed_height,imputed_duration,imputed_speed,intensity_level,imputed_inversion_count,type_Steel,type_Wood,design_Bobsled,design_Flying,...,imputed_restraints_Single position lap barHeadrest,imputed_restraints_Single position lap barHeadrestSeatbelt,imputed_restraints_Single position lap barSeatbelt,imputed_restraints_T-Bar,imputed_restraints_T-BarSeatbelt,imputed_restraints_UNKNOWN,imputed_intensity_Extreme,imputed_intensity_Family,imputed_intensity_Kiddie,imputed_intensity_Thrill
id,,,,,,,,,,,,,,,,,,,,,
14116.0,1.446324,1.383226,1.321124,1.477463,3,0.0,False,True,False,False,...,False,False,False,True,False,False,True,False,False,False
16607.0,-0.468851,2.338803,0.990010,2.082367,3,1.0,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
108.0,0.927266,1.179932,0.456765,1.511069,3,1.0,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
7.0,1.952626,2.221698,2.460782,1.947944,3,4.0,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
11437.0,3.085625,5.547481,3.189515,4.031501,3,0.0,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [90]:
ratings = rated['rating'].values.reshape(-1, 1)  # already done, likely
user_profile = (rated_features * ratings).sum(axis=0) / ratings.sum()

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare all coasters to your profile
similarities = cosine_similarity(X_final, user_profile.values.reshape(1, -1)).flatten()

# Add to the original coaster DataFrame (if you still have it)
roller_coaster_df['similarity'] = similarities

In [101]:
rated_ids = set(user_ratings_df[user_ratings_df['rating'].notnull()]['roller_coaster_id'])
unrated_coasters = roller_coaster_df[~roller_coaster_df.index.isin(rated_ids)]
top_recommendations = unrated_coasters.sort_values(by='similarity', ascending=False)
top_recommendations = roller_coaster_df.sort_values(by='similarity', ascending=False)

# Show top 10 (you can include name, type, intensity, etc.)
top_recommendations[['name', 'similarity', 'type', 'design', 'imputed_intensity','amusement_park']].head(100)

,name,similarity,type,design,imputed_intensity,amusement_park
id,,,,,,
110.0,Loch Ness Monster,0.973778,Steel,Sit Down,Extreme,Busch Gardens Williamsburg
10831.0,Nefeskesen,0.973470,Steel,Sit Down,Extreme,Vialand
16812.0,Pantheon,0.968051,Steel,Sit Down,Extreme,Busch Gardens Williamsburg
7395.0,Red Fire,0.967910,Steel,Sit Down,Extreme,Viaport Marina Tema Park
4147.0,iSpeed,0.967849,Steel,Sit Down,Extreme,Mirabilandia
...,...,...,...,...,...,...
17412.0,Abyssus,0.914061,Steel,Sit Down,Extreme,Energylandia
21294.0,Flash Coaster,0.913911,Steel,Sit Down,Extreme,Aquashow Family Park
12083.0,Baron 1898,0.913881,Steel,Sit Down,Extreme,Efteling
